<a href="https://colab.research.google.com/github/Sulaiman29/IntelliBrief/blob/main/Training_Vectorisation_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook is used to do the pre-processing tasks.

In [5]:
!apt install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [6]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [7]:
!unzip Dataset.zip -d Dataset

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/BBC News Summary/Summaries/sport/363.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/364.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/365.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/366.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/367.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/368.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/369.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/370.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/371.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/372.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/373.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/374.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/375.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport/376.txt  
  inflating: Dataset/BBC News Summary/Summaries/sport

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os

In [10]:
domains = ['business', 'entertainment', 'politics', 'sport', 'tech']
base_path = 'Dataset/BBC News Summary/News Articles/'
dataset_string = ''
exceptions = []
for domain in domains:
  domain_path = base_path + domain + '/'
  for name in os.listdir(domain_path):
    full_path = domain_path + name
    file = open(full_path, "r")
    try:
      dataset_string+=file.read()
    except:
      exceptions.append(full_path)

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
sentences = sent_tokenize(dataset_string)
og_sentences = []
for sent in sentences:
  splitting = sent.split('\n')
  for split in splitting:
    split = split.strip()
    og_sentences.append(split)

og_sentences

['US gives foreign firms extra time',
 '',
 'Foreign firms have been given an extra year to meet tough new corporate governance regulations imposed by the US stock market watchdog.',
 'The Securities and Exchange Commission has extended the deadline to get in line with the rules until 15 July 2006.',
 'Many foreign firms had protested that the SEC was imposing an unfair burden.',
 'The new rules are the result of the Sarbanes-Oxley Act, part of the US clean-up after corporate scandals such as Enron and Worldcom.',
 'Section 404 of the Sox Act, as the legislation is nicknamed, calls for all firms to certify that their financial reporting is in line with US rules.',
 'Big US firms already have to meet the requirements, but smaller ones and foreign-based firms which list their shares on US stock markets originally had until the middle of this year.',
 "Over the past few months, delegations of European and other business leaders have been heading to the SEC's Washington DC headquarters to 

In [14]:
word_tokenization = [word_tokenize(x) for x in og_sentences]
stop_words_removal = [list(filter(lambda x: x not in stopwords.words('english'), sent)) for sent in word_tokenization]

In [15]:
porter = PorterStemmer()

In [16]:
stemmed_words = [list(map(porter.stem, sent)) for sent in stop_words_removal]

In [17]:
stemmed_sentences = [" ".join(sentence) for sentence in stemmed_words]

In [18]:
for i in range(len(stemmed_sentences)):
  stemmed_sentences[i] = stemmed_sentences[i].replace(' ,', ',')
  stemmed_sentences[i] = stemmed_sentences[i].replace(' .', '.')
  stemmed_sentences[i] = stemmed_sentences[i].replace('``', '')
  stemmed_sentences[i] = stemmed_sentences[i].replace('\'\'', '')
  stemmed_sentences[i] = stemmed_sentences[i].replace('\'s', '')
  stemmed_sentences[i] = stemmed_sentences[i].replace('\'', '')
  stemmed_sentences[i] = stemmed_sentences[i].replace(':', '')

In [19]:
while '' in stemmed_sentences:
  stemmed_sentences.remove('')

In [20]:
dataset_string = " ".join(stemmed_sentences)

In [21]:
f = open("data.txt", "x")
f.write(dataset_string)

3353460

In [22]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313506 sha256=a090ea952da1cba1cdf1d1926ca063d8c4b4bf9c295dcf254831edd8b67db3e7
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [23]:
import fasttext

In [24]:
model_skipgram = fasttext.train_unsupervised('data.txt', model='skipgram')

In [25]:
model_cbow = fasttext.train_unsupervised('data.txt', model='cbow')

In [26]:
model_skipgram.save_model("model_skipgram.bin")

In [27]:
model_cbow.save_model("model_cbow.bin")

In [28]:
model_skipgram.words

['the',
 'said',
 'i',
 '-',
 'mr',
 'would',
 '(',
 'it',
 'new',
 'also',
 'peopl',
 'but',
 'one',
 'us',
 'year',
 '%',
 'he',
 'could',
 'said.',
 'use',
 'last',
 'say',
 'make',
 'first',
 'game',
 '$',
 'nt',
 ')',
 '.',
 'like',
 'two',
 'get',
 'take',
 'go',
 'we',
 'govern',
 'time',
 'play',
 'a',
 'in',
 'world',
 'film',
 'uk',
 'compani',
 'work',
 'want',
 'best',
 'told',
 'firm',
 'show',
 'back',
 'includ',
 'number',
 'mani',
 'made',
 'music',
 'win',
 'plan',
 'come',
 'next',
 'labour',
 'bbc',
 'look',
 'set',
 'three',
 'need',
 'way',
 'expect',
 'market',
 'parti',
 'minist',
 'think',
 'year.',
 'report',
 '),',
 'servic',
 'help',
 'good',
 'call',
 'home',
 'million',
 'see',
 'sale',
 'player',
 'nation',
 'still',
 'top',
 'may',
 'thi',
 'start',
 'well',
 'countri',
 'mobil',
 'sinc',
 'ad',
 'gener',
 'part',
 'much',
 'second',
 'put',
 'they',
 'offer',
 'right',
 'record',
 'public',
 'and',
 'end',
 'claim',
 'there',
 'tri',
 'technolog',
 'blai

In [29]:
model_cbow.get_word_vector('combin')

array([ 0.16843517,  0.15996848, -0.1137258 , -0.17888448,  0.102006  ,
       -0.01720394,  0.03323021, -0.01247154, -0.02091876, -0.30609855,
        0.0521182 ,  0.10062538,  0.25139213, -0.08952529,  0.08565051,
        0.05524563, -0.04428936, -0.07162308, -0.04886711, -0.03039474,
        0.03946431, -0.03997637,  0.03559269,  0.10859244, -0.16490488,
       -0.23505537, -0.07466875, -0.16590442, -0.33867648,  0.13623127,
       -0.41319495,  0.14944707, -0.14981614,  0.06251105, -0.4009151 ,
        0.04037252, -0.3188799 ,  0.05993445, -0.20494641,  0.00975438,
        0.10108875, -0.09781573, -0.20584106, -0.1176497 ,  0.5715494 ,
       -0.07123365, -0.11422287, -0.05523985, -0.12359191,  0.2743065 ,
       -0.07366823, -0.57274485,  0.08494669,  0.3581912 , -0.3034014 ,
       -0.19531922,  0.24732031, -0.11021809,  0.30936426, -0.25087398,
       -0.19207406, -0.19775508,  0.11061727,  0.40458897,  0.04306715,
       -0.25392362, -0.26933822,  0.30541754,  0.08651769, -0.06

In [30]:
from scipy import spatial
dataSetI = model_skipgram.get_word_vector('Razi is going to the mall')
dataSetII = model_skipgram.get_word_vector('Spiderman saved gwen stefani')
result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
result

np.float32(0.803819)